Цель: Получить резюме кандидатов из компаний, в которых наблюдается массовый отток сотрудников.

Задача: Реализовать скрипт в Python, возвращающий список резюме сооискателей по принципу совпадения текущего/последнего места работы кандидатов.
Необходимо определить компании, у которых текущие/бывшие сотрудники сейчас размещают резюме. И вывести резюме в сортировке по компаниям, где таких кандидатов больше всего.
Скрипт должен поддерживать конфигурирование основных параметров поиска(ключевые слова, глубину, проф.область, город итд. Некоторые параметры, такие как проф.область="ит.телеком" и город="Москва" можно преднастроить по умолчанию).

К сожалению, поиск резюме в API отсутсвует, доступен лишь просмотр резюме через метод https://api.hh.ru/resumes/
Для поиска предлагается использовать подход с парсингом сайта и получением идентификаторов резюме с использованием библиотек Selenium и BeatifualSoap (на основе сформированного поиского запроса к web-версии сайта, например https://hh.ru/search/resume?text=JAVA&st=resumeSearch&logic=normal&pos=full_text&exp_period=all_time&exp_company_size=any&exp_industry=any&area=113&relocation=living_or_relocation&salary_from=&salary_to=&currency_code=RUR&experience=between1And3&education=higher&age_from=&age_to=&gender=unknown&employment=full&schedule=fullDay&language=eng.b1&order_by=relevance&search_period=30&items_on_page=100 ) .
Далее идентификаторы можно просматривать в цикле и формировать из них массив данных (предварительно авторизовавшись с использованием токена корпоративной учетной записи).


In [1]:
import pandas as pd
import json

data = pd.read_json('data/data4.json')
data.head()

,title,href,last_work_plase
0,Начинающий гейм-дизайнер/Junior Game Developer,/resume/e5096a4e0007dbf47b0039ed1f38766664556e...,"ООО ""МОИСС"""
1,Программист 1С,/resume/3b24d28c00055dc3910039ed1f6f48426a7932...,"Полимерстрой, ТД, ООО"
2,Руководитель интернет-проектов / Менеджер инте...,/resume/464c6ecf0001e092050039ed1f55705a39496d...,"Велес Капитал, ИК"
3,Системный администратор\Инженер сц.,/resume/f156683e0001a7afb60039ed1f4b3478464745...,"ооо ""цто мастер""(штрих-м)"
4,Руководитель ИТ-подразделения,/resume/245b8f5a000007f4e50039ed1f736563726574...,"АйДи – Технологии управления, ООО"


In [3]:
data["href"] = 'https://hh.ru'+data["href"]

In [4]:
data.head()

,title,href,last_work_plase
0,Начинающий гейм-дизайнер/Junior Game Developer,https://hh.ruhttps://hh.ru/resume/e5096a4e0007...,"ООО ""МОИСС"""
1,Программист 1С,https://hh.ruhttps://hh.ru/resume/3b24d28c0005...,"Полимерстрой, ТД, ООО"
2,Руководитель интернет-проектов / Менеджер инте...,https://hh.ruhttps://hh.ru/resume/464c6ecf0001...,"Велес Капитал, ИК"
3,Системный администратор\Инженер сц.,https://hh.ruhttps://hh.ru/resume/f156683e0001...,"ооо ""цто мастер""(штрих-м)"
4,Руководитель ИТ-подразделения,https://hh.ruhttps://hh.ru/resume/245b8f5a0000...,"АйДи – Технологии управления, ООО"


In [5]:
count_company = data["last_work_plase"].value_counts()
df_count_company = pd.DataFrame({'last_work_plase':count_company.index, 'count':count_company.values})
df_count_company.head(30)

,last_work_plase,count
0,Берингер Ингельхайм,1
1,NEWMEN,1
2,"ООО ""Искусство Света""",1
3,Кворум,1
4,"ООО ""МОИСС""",1
5,HTMLAcademy,1
6,Кухня на районе,1
7,Рокетбанк,1
8,"АО ""Сарэкс""",1
9,"АйДи – Технологии управления, ООО",1


In [6]:
data_with_count = data.set_index('last_work_plase').join(df_count_company.set_index('last_work_plase'))
data_with_count.head()

,title,href,count
last_work_plase,,,
"ООО ""МОИСС""",Начинающий гейм-дизайнер/Junior Game Developer,https://hh.ruhttps://hh.ru/resume/e5096a4e0007...,1
"Полимерстрой, ТД, ООО",Программист 1С,https://hh.ruhttps://hh.ru/resume/3b24d28c0005...,1
"Велес Капитал, ИК",Руководитель интернет-проектов / Менеджер инте...,https://hh.ruhttps://hh.ru/resume/464c6ecf0001...,1
"ооо ""цто мастер""(штрих-м)",Системный администратор\Инженер сц.,https://hh.ruhttps://hh.ru/resume/f156683e0001...,1
"АйДи – Технологии управления, ООО",Руководитель ИТ-подразделения,https://hh.ruhttps://hh.ru/resume/245b8f5a0000...,1


In [7]:
sorted_data = data_with_count.sort_values(by=["count"], ascending=False)
sorted_data.to_csv('data_sort.csv', sep='\t', index=False)

In [8]:
sorted_data.head(10)

,title,href,count
last_work_plase,,,
"ООО ""МОИСС""",Начинающий гейм-дизайнер/Junior Game Developer,https://hh.ruhttps://hh.ru/resume/e5096a4e0007...,1
"Полимерстрой, ТД, ООО",Программист 1С,https://hh.ruhttps://hh.ru/resume/3b24d28c0005...,1
Глобалстрой-Инжиниринг,Ведущий специалист,https://hh.ruhttps://hh.ru/resume/15b493db0007...,1
HTMLAcademy,Frontend-разработчик (Junior),https://hh.ruhttps://hh.ru/resume/fc625ea80006...,1
CyberAnalytics,JavaScript React/Angular,https://hh.ruhttps://hh.ru/resume/074ffcee0007...,1
Кухня на районе,Специалист службы поддержки пользователей,https://hh.ruhttps://hh.ru/resume/2f8239010007...,1
NEWMEN,Менеджер по работе с клиентами,https://hh.ruhttps://hh.ru/resume/f9c923480004...,1
Кворум,Программист C#,https://hh.ruhttps://hh.ru/resume/210e263f0002...,1
Рокетбанк,Специалист службы поддержки пользователей,https://hh.ruhttps://hh.ru/resume/bcaa98430007...,1


In [9]:
sorted_data.to_csv('sorted_data_java.csv', sep='\t', index=False)

In [14]:
try:
    sorted_data.drop(["Индивидуальное предпринимательство / частная практика / фриланс"], inplace = True)
except:
    print("no that row")
#if (sorted_data["Индивидуальное предпринимательство / частная практика / фриланс"] != null):
#    sorted_data.drop(["Индивидуальное предпринимательство / частная практика / фриланс"], inplace = True)
#sorted_data.drop(["Фриланс"], inplace = True)
#sorted_data.drop(["Freelance"], inplace = True)
#sorted_data.drop(["фриланс"], inplace = True)

You can't divide by zero


In [15]:
sorted_data.head(107)

,title,href,count
last_work_plase,,,
"ООО ""МОИСС""",Начинающий гейм-дизайнер/Junior Game Developer,https://hh.ruhttps://hh.ru/resume/e5096a4e0007...,1
"Полимерстрой, ТД, ООО",Программист 1С,https://hh.ruhttps://hh.ru/resume/3b24d28c0005...,1
Глобалстрой-Инжиниринг,Ведущий специалист,https://hh.ruhttps://hh.ru/resume/15b493db0007...,1
HTMLAcademy,Frontend-разработчик (Junior),https://hh.ruhttps://hh.ru/resume/fc625ea80006...,1
CyberAnalytics,JavaScript React/Angular,https://hh.ruhttps://hh.ru/resume/074ffcee0007...,1
Кухня на районе,Специалист службы поддержки пользователей,https://hh.ruhttps://hh.ru/resume/2f8239010007...,1
NEWMEN,Менеджер по работе с клиентами,https://hh.ruhttps://hh.ru/resume/f9c923480004...,1
Кворум,Программист C#,https://hh.ruhttps://hh.ru/resume/210e263f0002...,1
Рокетбанк,Специалист службы поддержки пользователей,https://hh.ruhttps://hh.ru/resume/bcaa98430007...,1


In [16]:
sorted_data.to_csv('sorted_data_java.csv', sep='\t', index=False)